###**IMPORTING** **LIBRARIES**


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


###**IMPORTING DATASET AND PREPROCESSING**

In [ ]:
df = pd.read_csv("Dataset .csv")
mode_cuisine = df['Cuisines'].mode()[0]
df['Cuisines'].fillna(mode_cuisine, inplace=True)

###**CONVERTING ALL VALUES TO STRING**

In [ ]:
df['Cuisines'] = df['Cuisines'].astype(str)

###**ENCODING CATAGORICAL VALUES**

In [ ]:
label_encoders = {}
categorical_cols = ['Country Code', 'City', 'Locality', 'Cuisines', 'Currency', 'Has Table booking', 'Has Online delivery', 'Is delivering now', 'Price range', 'Rating color', 'Rating text']

In [ ]:
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

###**CRITERIA FOR RECOMMENDATION**

In [ ]:
criteria = {
    'Cuisine': 'Chinese',
    'Price Range': 2,
    'Has Table booking':'yes'
}

###**CONTENT BASED FILTERING**

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
cuisine_matrix = tfidf_vectorizer.fit_transform(df['Cuisines'].astype(str))


###**CALCULATING COSINE SIMILARITY**

In [ ]:
cuisine_similarity = cosine_similarity(cuisine_matrix, cuisine_matrix)

###**FUNCTION TO GET RESTAURANT RECOMMENDATION**

In [ ]:
df['Cuisines'] = df['Cuisines'].astype(str)

def get_recommendations(criteria):
    # Filter restaurants based on criteria
    filtered_df = df[(df['Cuisines'].str.contains(criteria['Cuisine'])) & (df['Price range'] == criteria['Price Range'])]
    filtered_indices = filtered_df.index
    similarity_scores = cuisine_similarity[filtered_indices].sum(axis=0)
    sorted_indices = similarity_scores.argsort()[::-1]
    recommended_restaurants = df.iloc[sorted_indices[:5]]
    return recommended_restaurants[['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating']]

###**TESTING THE SYSYTEM**

In [ ]:
sample_criteria = {'Cuisine': 'Korean', 'Price Range': 2}
recommendations = get_recommendations(sample_criteria)
print(recommendations)

                                Restaurant Name Cuisines  Price range  \
9550                   Walter's Coffee Roastery      331            1   
3188  Cake Central - Premier Cake Design Studio      186            0   
3186                                    Barista      331            1   
3185                          Anil Mishtan Wala     1262            0   
3184                                   Aka Saka      533            2   

      Aggregate rating  
9550               4.0  
3188               3.4  
3186               3.3  
3185               3.1  
3184               3.3  
